# Units to Unify

As I want to make sure each functions is producing something that I am looking for, the idea is to break from big to small to the unbreakbale parts. 

We starts from the input, I will make up some fake data in correct dimension, to start with:

## Input

### Volunteers' answer

what is q(t) looks like after computation? I need to make sure that this dim matches the y_train for yolo. 
but is this a right thing to do?
how about I check the y_train for lenet5 first, then check q(t), see how the original bcc works, then decide what to do. 


In [3]:
# confusion_matrix.py 

import numpy as np

def initialise_prior(n_classes, n_volunteers, alpha_diag_prior):
    """
    Create confusion matrix prior for every volunteer - the same prior for each volunteer
    :param n_classes: number of classes (int)
    :param n_volunteers: number of crowd members (int)
    :param alpha_diag_prior: prior for confusion matrices is assuming reasonable crowd members with weak dominance of a
    diagonal elements of confusion matrices, i.e. prior for a confusion matrix is a matrix of all ones where
    alpha_diag_prior is added to diagonal elements (float)
    :return: numpy nd-array of the size (n_classes, n_classes, n_volunteers)
    """
    alpha_volunteer_template = np.ones((n_classes, n_classes), dtype=np.float64) + alpha_diag_prior * np.eye(n_classes)
    return np.tile(np.expand_dims(alpha_volunteer_template, axis=2), (1, 1, n_volunteers))



In [13]:
alpha_0 = initialise_prior(2, 4, 0.1)
print(alpha_0)
print(alpha_0.shape)
print(len(alpha_0.shape)) # will be used in defining expected_log_Dirichlet_parameters(param)
print(alpha_0[:,:,2]) # all the same for each individual

# for each volunteer (in 4), the prior parameters of confusion matrix is the line above



[[[1.1 1.1 1.1 1.1]
  [1.  1.  1.  1. ]]

 [[1.  1.  1.  1. ]
  [1.1 1.1 1.1 1.1]]]
(2, 2, 4)
3
[[1.1 1. ]
 [1.  1.1]]


In [11]:
import scipy.special as ss


def expected_log_Dirichlet_parameters(param): #Eq.(16)
    # param: alpha_volunteers
    size = param.shape
    # shape:(J X L X W)

    result = np.zeros_like(param)

    if len(size) == 1:
        result = ss.psi(param) - ss.psi(np.sum(param))
        # ss.psi(): The logarithmic derivative of the gamma function evaluated at param
    elif len(size) == 2:
        result = ss.psi(param) - np.transpose(np.tile(ss.psi(np.sum(param, 1)), (size[1], 1)))
    elif len(size) == 3:
        for i in range(size[2]): # for each volunteer
            result[:, :, i] = ss.psi(param[:, :, i]) - \
                              np.transpose(np.tile(ss.psi(np.sum(param[:, :, i], 1)), (size[1], 1)))
    else:
        raise Exception('param can have no more than 3 dimensions')

    return result

ElogPi_volunteer = expected_log_Dirichlet_parameters(alpha_0)

# not important comment: the input len(input.shape) = 3 all the time, i dont see why we consider the first two cases.
# elif len(size==3), step by step:
#  step1 = np.sum(alpha_0[:, :, 0], 1) #summing over row of cm (2,)
#  step2 = ss.psi(step1) # digamma function (2,)
#  step3 = np.tile(step2, (2,1)) # make the shape of previous to (2,2)
#  step4 = ss.psi(alpha_0[:, :, 0]) # digamma each cm input (2,2)
#  step4 - np.transpose(step3) # get the result


In [65]:
print(ElogPi_volunteer)
print(ElogPi_volunteer.shape) # the same shape as alpha_0
print(ElogPi_volunteer[:,:,3])

[[[-0.90909091 -0.90909091 -0.90909091 -0.90909091]
  [-1.06255163 -1.06255163 -1.06255163 -1.06255163]]

 [[-1.06255163 -1.06255163 -1.06255163 -1.06255163]
  [-0.90909091 -0.90909091 -0.90909091 -0.90909091]]]
(2, 2, 4)
[[-0.90909091 -1.06255163]
 [-1.06255163 -0.90909091]]


In [ ]:
def expected_true_labels(X, nn_output, ElogPi_volunteer):
    N, W = X.shape  # N = Number of subjects, W = Number of volunteers.
    J = ElogPi_volunteer.shape[0]  # J = Number of classes
    L = ElogPi_volunteer.shape[1] # L = Number of classes used by volunteers
### comments: ElogPi_volunteer is the LHS of Eq. (16), the A_ji^(k)
    rho = np.copy(nn_output) # N X J logits

    for w in range(W):
        inds = np.where(X[:, w] > -1) #extract index for filled objects
        # the above line can be deleted as once the bbox is drawn, it must comes with a label. i.e. no missing data
        # not deleting maybe, keep it incase things went wrong
        rho[inds, :] = rho[inds, :] + np.transpose(ElogPi_volunteer[:, np.squeeze(X[inds, w]), w])

    rho = rho - np.transpose(np.tile(np.max(rho, 1), (J, 1)))

    q_t = np.exp(rho) / np.maximum(1e-60, np.transpose(np.tile(np.sum(np.exp(rho), 1), (J, 1))))
    q_t = np.maximum(1e-60, q_t)

    Njl = np.zeros((J, L, W), dtype=np.float64)
    for w in range(W):
        for l in range(L):
            inds = np.where(X[:, w] == l)[0]
            Njl[:, l, w] = np.sum(q_t[inds, :], 0)

    return q_t, Njl, rho




In [66]:
# NxW(5x4) as indiduals answers.
v_ans = np.array([[0, 1, 1, -1], [1, 1, 0, 0], [0,1,0,1], [0,0,1,1], [1,1,1,0]])
print(v_ans.shape)
print(v_ans)



(5, 4)
[[ 0  1  1 -1]
 [ 1  1  0  0]
 [ 0  1  0  1]
 [ 0  0  1  1]
 [ 1  1  1  0]]


In [43]:
# the dim of nn_output is N X J, 5 X 2 
fake_nn_output = np.array([[2.8, 7.6], [3.9, 8.3], [7.1,1.1], [6.0,7.2], [3.2,7.2]])
print(fake_nn_output.shape)

(5, 2)


In [69]:
# test me 
indss = np.where(v_ans[:, 3] > -1)
print(indss)
check_this = np.squeeze(v_ans[indss, 3])
print(check_this)
update_rho = fake_nn_output[indss, :] = fake_nn_output[indss, :] + np.transpose(ElogPi_volunteer[:, check_this, 3])
print(update_rho)
print(update_rho.shape)

(array([1, 2, 3, 4]),)
[0 1 1 0]
[[[ 2.08181818  6.17489673]
  [ 4.97489673 -0.71818182]
  [ 3.87489673  5.38181818]
  [ 1.38181818  5.07489673]]]
(1, 4, 2)


In [71]:
# test me 
indsss = np.where(v_ans[:, 0] > -1)
print(indsss)
check_that = np.squeeze(v_ans[indsss, 0])
print(check_that)
update_rhoo = fake_nn_output[indsss, :] = fake_nn_output[indsss, :] + np.transpose(ElogPi_volunteer[:, check_that, 0])
print(update_rhoo)
print(update_rhoo.shape)

(array([0, 1, 2, 3, 4]),)
[0 1 0 0 1]
[[[ 1.89090909  6.53744837]
  [ 1.01926655  5.26580582]
  [ 4.06580582 -1.78073345]
  [ 2.96580582  4.31926655]
  [ 0.31926655  4.16580582]]]
(1, 5, 2)


In [ ]:
for w in range(W):
        inds = np.where(X[:, w] > -1) #extract index for filled objects
        # the above line can be deleted as once the bbox is drawn, it must comes with a label. i.e. no missing data
        # not deleting maybe, keep it incase things went wrong
        rho[inds, :] = rho[inds, :] + np.transpose(ElogPi_volunteer[:, np.squeeze(X[inds, w]), w])
        
        
        

okay firstly we figure out how it going to be look like for the original problem
then expand the dim for yolo output
19*19*(3*(2))

now let's assume N=5, we have 5 iamges
then we have NxW(5x4) as indiduals answers.

we dont care about -1 entries, why? if a bounding box is drawn it must comes with label
that is to say: inds = np.where(X[:, w] > -1) can be disabled


rho[inds, :] = rho[inds, :] + np.transpose(ElogPi_volunteer[:, np.squeeze(X[inds, w]), w])

rho[inds, :] for volunteer 4, there is only four images has answer, the position one is missing, so this firstly take out the (4x2) from nn_output.

the answer of the four entries (array([1, 2, 3, 4]),) is [3 1 0 2]. thus,

